#Introduction

Tire selection plays a key role in ensuring road safety, especially in extreme weather conditions. The type of tires used on your vehicle depends on the season and weather conditions. Summer tires offer excellent traction in hot, dry conditions, while winter tires offer better traction in cold, ice, and snow. All-season tires are designed to perform well in both summer and winter. 
 
The aim of this project is to develop a model for identifying the tire type, which distinguishes between summer, winter and all-season tires in particular. The data set used to train and test the model will contain images of tires with different tire wear patterns. 
 
 The development of an accurate tire type recognition model could make a significant contribution to road safety by providing drivers with an automated means of ensuring they are using the correct tires for  current weather conditions. This project aims to harness the power of deep learning to create an efficient and effective tire type recognition system.

## Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Opony_Computer_Vision

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Opony_Computer_Vision


In [2]:
import requests
from bs4 import BeautifulSoup
import time
import  random
import os
from io import BytesIO
from PIL import Image
import cv2
import numpy as np

## Scraping test

In [ ]:
summer = '1'
winter = '2'
both = '3'
page = '1'


url = f'https://www.24opony.pl/szukaj.html?f=1&idproducent=0&idsezon={summer}&idrozmiar=0&idpojazd=1&czyxl=0&czyrf=0&inpre=0&cod=0&cdo=0&strona={page}&idmodel=0'

content = requests.get(url).content
soup = BeautifulSoup(content, 'html.parser')

In [ ]:
images = ['https://www.24opony.pl' + image.find('img')['src'] for image in soup.find_all('a', {'rel':'prettyPhoto'})]

In [ ]:
images[0].split('/')[-1].split('.')[0]

'n_4164_s'

In [ ]:
images

In [ ]:
content = requests.get(images[0]).content

In [ ]:
img  = Image.open(BytesIO(content))

In [ ]:
img.max()

AttributeError: ignored

## Scraping

In [ ]:
def get_page(season, page_number):
  url = f'https://www.24opony.pl/szukaj.html?f=1&idproducent=0&idsezon={season}&idrozmiar=0&idpojazd=1&czyxl=0&czyrf=0&inpre=0&cod=0&cdo=0&strona={page_number}&idmodel=0'
  try:
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')
    time.sleep(0.5)
    return content
  except Exception as e:
    print(f'Error: {e}')

def get_urls(content):
  soup = BeautifulSoup(content, 'html.parser')
  return set(['https://www.24opony.pl' + image.find('img')['src'] for image in soup.find_all('a', {'rel':'prettyPhoto'}) if not image.find('img')['src'].split('/')[-1].split('.')[0].startswith('brak_zdjecia')])


def sharpen_image(func):
  def wrapper(url):
    img = func(url)
    new_img = cv2.filter2D(img, -1, kernel=np.array([[0, -1, 0],
                                                [-1, 5, -1],
                                                [0, -1, 0]]))
    return new_img
  return wrapper
    
@sharpen_image
def get_image(url):
  try:
    content = requests.get(url).content
    img = Image.open(BytesIO(content))
    img = np.array(img)
    return img
  except Exception as e:
    print(f'Error: {e}')

def save_image(img, url,season, path):
  file_name = url.split('/')[-1].split('.')[0] + '.jpg'
  os.makedirs(f'{path}/{season}', exist_ok=True)
  cv2.imwrite(f'{path}/{season}/{file_name}', img)

def get_data(season, start_page, end_page, path):
  for page_number in range(start_page, end_page):
    content = get_page(season, page_number + 1)
    urls = get_urls(content)

    for i, url in enumerate(urls):
      img = get_image(url)      
      save_image(img, url, season, path)
      time.sleep(1.0/100)
      if  i % 5 == 0:
        print(f'Download {i} images from page {page_number}/{end_page}')



In [ ]:
#dodaj lsitę z linkami, tak że gdy się link powtarza to skipuje

get_data(1, 1, 400, 'data/train')
get_data(2, 1, 400, 'data/train')
get_data(3, 1, 400, 'data/train')

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Download 15 images from page 9/400
Download 20 images from page 9/400
Download 0 images from page 10/400
Download 5 images from page 10/400
Download 10 images from page 10/400
Download 15 images from page 10/400
Download 0 images from page 11/400
Download 5 images from page 11/400
Download 10 images from page 11/400
Download 15 images from page 11/400
Download 20 images from page 11/400
Download 0 images from page 12/400
Download 5 images from page 12/400
Download 10 images from page 12/400
Download 15 images from page 12/400
Download 0 images from page 13/400
Download 5 images from page 13/400
Download 10 images from page 13/400
Download 15 images from page 13/400
Download 0 images from page 14/400
Download 5 images from page 14/400
Download 10 images from page 14/400
Download 15 images from page 14/400
Download 20 images from page 14/400
Download 0 images from page 15/400
Download 5 images from page 15/400
Download 10 i

In [3]:
def get_page(season, page_number):
  url = f'https://www.24opony.pl/szukaj.html?f=1&idproducent=0&idsezon={season}&idrozmiar=0&idpojazd=1&czyxl=0&czyrf=0&inpre=0&cod=0&cdo=0&strona={page_number}&idmodel=0'
  try:
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')
    time.sleep(0.5)
    return content
  except Exception as e:
    print(f'Error: {e}')

def get_urls(content):
  soup = BeautifulSoup(content, 'html.parser')
  return set(['https://www.24opony.pl' + image['href'] for image in soup.find_all('a', {'rel':'prettyPhoto'}) if not image['href'].split('/')[-1].split('.')[0].startswith('brak_zdjecia')])


def sharpen_image(func):
  def wrapper(url):
    img = func(url)
    new_img = cv2.filter2D(img, -1, kernel=np.array([[0, -1, 0],
                                                [-1, 5, -1],
                                                [0, -1, 0]]))
    return new_img
  return wrapper
    
@sharpen_image
def get_image(url):
  try:
    content = requests.get(url).content
    img = Image.open(BytesIO(content))
    img = np.array(img)
    return img
  except Exception as e:
    print(f'Error: {e}')

def save_image(img, url,season, path):
  file_name = url.split('/')[-1].split('.')[0] + '.jpg'
  os.makedirs(f'{path}/{season}', exist_ok=True)
  cv2.imwrite(f'{path}/{season}/{file_name}', img)

def get_data(season, start_page, end_page, path):
  for page_number in range(start_page, end_page):
    content = get_page(season, page_number + 1)
    urls = get_urls(content)

    for i, url in enumerate(urls):
      img = get_image(url)      
      save_image(img, url, season, path)
      time.sleep(1.0/100)
      if  i % 5 == 0:
        print(f'Download {i} images from page {page_number}/{end_page}')


In [ ]:
# get_data(1, 1, 400, 'data/train_various_sizes')
# get_data(2, 1, 400, 'data/train_various_sizes')
get_data(3, 1, 400, 'data/train_various_sizes')

Download 0 images from page 1/400
Download 5 images from page 1/400
Download 10 images from page 1/400
Download 15 images from page 1/400
Download 0 images from page 2/400
Download 5 images from page 2/400
Download 10 images from page 2/400
Download 15 images from page 2/400
Download 0 images from page 3/400
Download 5 images from page 3/400
Download 10 images from page 3/400
Download 15 images from page 3/400
Download 0 images from page 4/400
Download 5 images from page 4/400
Download 10 images from page 4/400
Download 15 images from page 4/400
Download 0 images from page 5/400
Download 5 images from page 5/400
Download 10 images from page 5/400
Download 15 images from page 5/400
Download 0 images from page 6/400
Download 5 images from page 6/400
Download 10 images from page 6/400
Download 15 images from page 6/400
Download 0 images from page 7/400
Download 5 images from page 7/400
Download 10 images from page 7/400
Download 15 images from page 7/400
Download 0 images from page 8/400


#Model

In [ ]:
test_size = 0.2

# for sub_type in ['sneakersy', 'baleriny', 'szpilki', 'bieganie', 'trampki']:
#   os.makedirs(path + '/test' +f'/{sub_type}', exist_ok=True)
#   file_list = os.listdir(path + '/train/' + sub_type)
#   number_of_samples = int(len(file_list) * test_size)

#   for file_name in random.sample(file_list, number_of_samples):
#     os.rename(path +'/train/' + sub_type + '/' + file_name,
#               path +'/test/' + sub_type + '/' + file_name)